In [10]:
import glob
import os
import sys
import time
import random

import carla

import torch

import carla
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from PIL import Image

from agents.navigation.basic_agent import BasicAgent

import gym
import gym_carla

import torch.nn as nn
from itertools import count
import torch.nn.functional as F

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [21]:
# parameters for the gym_carla environment
params = {
    'number_of_vehicles': 2,
    'number_of_walkers': 0,
    'display_size': 256,  # screen size of bird-eye render
    'max_past_step': 1,  # the number of past steps to draw
    'dt': 0.1,  # time interval between two frames
    'discrete': True,  # whether to use discrete control space
    'discrete_acc': [2.0],  # discrete value of accelerations
    'discrete_steer': [-0.3, 0.0, 0.3],  # discrete value of steering angles
    'continuous_accel_range': [-3.0, 3.0],  # continuous acceleration range
    'continuous_steer_range': [-0.3, 0.3],  # continuous steering angle range
    'ego_vehicle_filter': 'vehicle.lincoln*',  # filter for defining ego vehicle
    'port': 3000,  # connection port
    'town': 'Town04',  # which town to simulate
    'task_mode': 'random',  # mode of the task, [random, roundabout (only for Town03)]
    'max_time_episode': 200,  # maximum timesteps per episode
    'max_waypt': 12,  # maximum number of waypoints
    'obs_range': 32,  # observation range (meter)
    'lidar_bin': 0.125,  # bin size of lidar sensor (meter)
    'obs_size': 128, #obs image sizes
    'd_behind': 12,  # distance behind the ego vehicle (meter)
    'out_lane_thres': 1.8,  # threshold for out of lane
    'desired_speed': 5,  # desired speed (m/s)
    'max_ego_spawn_times': 20,  # maximum times to spawn ego vehicle
    'display_route': True,  # whether to render the desired route
    'pixor_size': 64,  # size of the pixor labels
    'pixor': False,  # whether to output PIXOR observation
    'routes':None,
    'weather':'WetCloudySunset',
    'Collect_Data':False
}

# Set gym-carla environment
env = gym.make('carla-v0', params=params)

connecting to Carla server...
Carla server connected!
WeatherParameters(cloudiness=60.000000, cloudiness=60.000000, precipitation=0.000000, precipitation_deposits=50.000000, wind_intensity=10.000000, sun_azimuth_angle=-1.000000, sun_altitude_angle=15.000000, fog_density=2.000000, fog_distance=0.750000, fog_falloff=0.100000, wetness=0.000000, scattering_intensity=1.000000, mie_scattering_scale=0.030000, rayleigh_scattering_scale=0.033100)


In [22]:
class PerceptionNet(nn.Module):

    def __init__(self):
        super(PerceptionNet,self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.conv4 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        
        self.conv5 = nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        
        self.conv6 = nn.Conv2d(512, 64, kernel_size=4, stride=1)
        
        self.conv7 = torch.nn.ConvTranspose2d(64,512, kernel_size =4, stride=1)
        self.bn6 = nn.BatchNorm2d(512)
        
        self.conv8 = torch.nn.ConvTranspose2d(512,256, kernel_size =4, stride=2, padding=1)
        self.bn7 = nn.BatchNorm2d(256)
        
        self.conv9 = torch.nn.ConvTranspose2d(256,128, kernel_size =4, stride=2, padding=1)
        self.bn8 = nn.BatchNorm2d(128)
        
        self.conv10 = torch.nn.ConvTranspose2d(128,64, kernel_size =4, stride=2, padding=1)
        self.bn9 = nn.BatchNorm2d(64)
        
        self.conv11 = torch.nn.ConvTranspose2d(64,32, kernel_size =4, stride=2, padding=1)
        self.bn10 = nn.BatchNorm2d(32)
        
        self.conv12 = torch.nn.ConvTranspose2d(32,23, kernel_size =4, stride=2,padding=1)
        
            
    def encode(self, x):
        x = F.leaky_relu(self.conv1(x),negative_slope=0.02)
        x = F.leaky_relu(self.bn2(self.conv2(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn3(self.conv3(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn4(self.conv4(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn5(self.conv5(x)),negative_slope=0.02)
        return self.conv6(x)
    
    def decode(self, x):
        x = F.leaky_relu(self.bn6(self.conv7(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn7(self.conv8(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn8(self.conv9(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn9(self.conv10(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn10(self.conv11(x)),negative_slope=0.02)
        return F.log_softmax(self.conv12(x),dim=1)
    
    def forward(self, x):
        x = x.to(device, dtype=torch.float32)
        latent = self.encode(x)
        out = self.decode(latent)
        return out, latent

In [14]:
class PerceptionNet(nn.Module):

    def __init__(self):
        super(PerceptionNet,self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=4, stride=2, padding=1)
        
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        
        self.conv3 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        self.conv4 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        
        self.conv5 = nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        
        self.conv6a = nn.Conv2d(512, 64, kernel_size=4, stride=1)
        self.conv6b = nn.Conv2d(512, 64, kernel_size=4, stride=1)
        
        self.conv7 = torch.nn.ConvTranspose2d(64,512, kernel_size =4, stride=1)
        self.bn6 = nn.BatchNorm2d(512)
        
        self.conv8 = torch.nn.ConvTranspose2d(512,256, kernel_size =4, stride=2, padding=1)
        self.bn7 = nn.BatchNorm2d(256)
        
        self.conv9 = torch.nn.ConvTranspose2d(256,128, kernel_size =4, stride=2, padding=1)
        self.bn8 = nn.BatchNorm2d(128)
        
        self.conv10 = torch.nn.ConvTranspose2d(128,64, kernel_size =4, stride=2, padding=1)
        self.bn9 = nn.BatchNorm2d(64)
        
        self.conv11 = torch.nn.ConvTranspose2d(64,32, kernel_size =4, stride=2, padding=1)
        self.bn10 = nn.BatchNorm2d(32)
        
        self.conv12 = torch.nn.ConvTranspose2d(32,13, kernel_size =4, stride=2,padding=1)
        
            
    def encode(self, x):
        x = F.leaky_relu(self.conv1(x),negative_slope=0.02)
        x = F.leaky_relu(self.bn2(self.conv2(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn3(self.conv3(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn4(self.conv4(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn5(self.conv5(x)),negative_slope=0.02)
        return self.conv6a(x)

    
    def decode(self, x):
        x = F.leaky_relu(self.bn6(self.conv7(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn7(self.conv8(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn8(self.conv9(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn9(self.conv10(x)),negative_slope=0.02)
        x = F.leaky_relu(self.bn10(self.conv11(x)),negative_slope=0.02)
        return torch.sigmoid(self.conv12(x))

    def reparameterize(self,mu,logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        latent_sample = mu + eps*std
        return latent_sample
    
    def forward(self, x):
        x = x.to(device, dtype=torch.float32)
        latent = self.encode(x)
        #latent = self.reparameterize(mu,logvar)
        out = self.decode(latent)
        return out, latent

In [15]:
class DQN(nn.Module):

    def __init__(self, outputs):
        super(DQN, self).__init__()
        
        self.lin1 = nn.Linear(9,80)
        self.lin2 = nn.Linear(80,50)
        self.lin3 = nn.Linear(50,25)
        self.lin4 = nn.Linear(25,15)
        self.lin5 = nn.Linear(15,8)
        self.lin6 = nn.Linear(8,3)

    def forward(self, x):
        x = x.to(device)
        x = F.relu(self.lin1(x))
        x = F.relu(self.lin2(x))
        x = F.relu(self.lin3(x))
        x = F.relu(self.lin4(x))
        x = F.relu(self.lin5(x))
        x = self.lin6(x)
        
        return x.view(x.size(0), -1)

In [6]:
model = PerceptionNet()
model.to(device)
model.load_state_dict(torch.load('./AE_params/model_44.best'))
model.eval()

vis_net = DQN(3).to(device)
vis_net.load_state_dict(torch.load('./model_params_CL/model_7.best'))
vis_net.eval()

DQN(
  (lin1): Linear(in_features=9, out_features=80, bias=True)
  (lin2): Linear(in_features=80, out_features=50, bias=True)
  (lin3): Linear(in_features=50, out_features=25, bias=True)
  (lin4): Linear(in_features=25, out_features=15, bias=True)
  (lin5): Linear(in_features=15, out_features=8, bias=True)
  (lin6): Linear(in_features=8, out_features=3, bias=True)
)

In [6]:
from agents.navigation.basic_agent import BasicAgent


In [23]:
for i in [(False,1500),(True,200)]:
    num_episodes = i[1]
    min_overall_loss = 1000
    data = DataCollector(env,test=i[0])
    for i_episode in range(num_episodes):
        rewards = 0
        # Initialize the environment and state
        obs = env.reset()
        agent = BasicAgent(env.ego)
        agent.set_target_speed(10)
        #ego_dir retirves the distance and angle from vehicle to nearest waypoint
        ego_location = env.ego.get_location()
        ego_dir = gym_carla.envs.misc.get_lane_dis(env.waypoints,ego_location.x,ego_location.y)
        #pos gets a distanc d and array w which has to be seperated out in below line
        ego_pos = np.asarray((ego_dir[0],ego_dir[1][0],ego_dir[1][1]),dtype=np.float32)
        state = np.concatenate((ego_pos,np.zeros(6)))

        #inital state with previous action for data collection consistency
        action = np.array([0])
        reward = 0

        #ego_location = env.ego.get_location()
        #end = get_lane_dis(env.waypoints,ego_location.x,ego_location.y)
        #agent.set_destination(carla.Location(*end))

        data.collect_step(obs, state, reward, action)

        state = torch.tensor(state)
        loss = episode_loss = 1000
        for t in count():
            with torch.no_grad():
                action = random.randint(0,2)
            env.show_images(np.asarray(generate_semantic_im(obs['camera'])))
            next_obs, reward, done, info  = env.step(action)
            rewards += reward

            #pos gets a distanc d and array w which has to be seperated out in below line
            pos = np.asarray((info['position'][0],info['position'][1][0],info['position'][1][1]))
            ang = np.asarray(info['angular_vel'])
            acc = np.asarray(info['acceleration'])
            steer = np.asarray(info['steer'])
            next_state = np.concatenate((pos, ang, acc, steer), axis=None)
            
            #collect every other image
            data.collect_step(obs,state,reward,np.array([action]))
            
            #update state
            state = torch.tensor(next_state)
            obs = next_obs
            #obs = torch.tensor(next_obs)
            
            
            if done:
                data.stop_recording()
                break
print('Done')

In [7]:
def get_lane_dis(waypoints, x, y):
  """
  Calculate distance from (x, y) to waypoints.
  :param waypoints: a list of list storing waypoints like [[x0, y0], [x1, y1], ...]
  :param x: x position of vehicle
  :param y: y position of vehicle
  :return: a tuple of the distance and the closest waypoint orientation 
  :added - and waypoint
  """
  dis_min = 1000
  dis_sec = 999
  waypt = waypoints[0]
  for pt in waypoints:
    d = np.sqrt((x-pt[0])**2 + (y-pt[1])**2)
    if d < dis_min:
      dis_min = d
      waypt1 =pt
    elif d > dis_min and d < dis_sec:
      dis_sec = d
      waypt2=pt
  return waypt1, waypt2


In [33]:
agent.run_step().steer

-0.800000011920929

In [89]:
print(get_lane_dis(env.waypoints,ego_location.x,ego_location.y))

([-4.786291122436523, -82.82028198242188, 90.47937774658203], [-4.828125, -77.8204574584961, 90.47937774658203])


In [77]:
print(env.ego.get_location())

Location(x=-97.393295, y=-78.767982, z=0.012080)


In [56]:
print(env.waypoint_queue.pop()[0].transform.location)
print(env.waypoint_queue.popleft()[0].transform.location)

Location(x=-76.864464, y=-105.633743, z=0.000015)
Location(x=-9.575274, y=58.183372, z=0.034874)


In [16]:
agent.run_step().steer

-0.0020483587868511677

In [16]:
import time
import os
class DataCollector():
    def __init__(self, env, recorder = False, test=False):
        self.timestamp = time.strftime("%a, %d %b %Y %H:%M:%S")
        if not env:
            raise Exception("Please provide a carla gym env object")
        self.recorder = recorder
        
        dir_str = './Datasets/'+env.weather+'/'+env.town
        
        #collect test set
        if test:
            dir_str = dir_str +'/test'
            
        self.rgb_dir = dir_str+'/RGB/'
        self.sem_dir = dir_str+'/Semantic/'
        self.state_dir = dir_str+'/States/'
        self.recording_dir = dir_str+'/Recordings/'
        if not os.path.isdir(dir_str):
            os.makedirs(dir_str)
            os.makedirs(self.rgb_dir)
            os.makedirs(self.sem_dir)
            os.makedirs(self.state_dir)
            os.makedirs(self.recording_dir)
        
        if self.recorder:
            self.record()
    
            
    
    def collect_step(self, obs, state, reward, action):
        self.timestamp = time.strftime("%a, %d %b %Y %H:%M:%S")
        # save image data 
        im = Image.fromarray(obs['camera'])
        im.save(self.rgb_dir+self.timestamp+".jpeg")
        np.save(self.sem_dir+self.timestamp,obs['semantic'][:, :, :1])
        #convert tensor to numpy array and append action to state vector
        if torch.is_tensor(state):
            state = state.cpu()
            state = state.numpy()
        state = np.append(state,action)
        np.save(self.state_dir+self.timestamp,state)
        
        
        
        
    def record(self):
        env.client.start_recorder(self.recording_dir+self.timestamp+".log")

    def stop_recording(self):
        if self.recorder:
            env.client.stop_recorder()


In [22]:
obs['camera'].dtype

dtype('uint8')

In [1]:
Image.fromarray(obs['camera'],'RGB')

NameError: name 'Image' is not defined

In [19]:
def recode_tags(sem_image):
    recode_dict = {0:0,1:1,2:2,3:3,4:4,5:5,6:6,7:7,8:8,9:9,
                    10:10,11:11,12:12,13:0,14:3,15:1,16:3,17:2,18:5,19:3,20:4,21:3,22:9
                    }
    for value in recode_dict.keys():
        sem_image[sem_image==value] = recode_dict[value]
    return sem_image

tag_convert_dict = {0:[70,130,180],
                   1:[70,70,70],
                   2:[100,40,40],
                   3:[55,90,80],
                   4:[220,20,60],
                   5:[153,153,153],
                   6:[157,234,50],
                   7:[128,64,128],
                   8:[244,35,232],
                   9:[107,142,35],
                   10:[0,0,142],
                   11:[102,102,156],
                   12:[220,220,0],
                   13:[70,130,180],
                   14:[81,0,81],
                   15:[150,100,100],
                   16:[230,150,140],
                   17:[180,165,180],
                   18:[250,170,30],
                   19:[110,190,160],
                   20:[170,120,50],
                   21:[45,60,150],
                   22:[145,170,100],
                  }

def replace(a):
    a = a.reshape(128,128)
    pic = np.zeros((128,128,3),dtype='uint8')
    for x, y in np.ndindex(a.shape):
        value = a[x,y]
        RGB_values = tag_convert_dict[value]
        pic[x,y,0] = RGB_values[0]
        pic[x,y,1] = RGB_values[1]
        pic[x,y,2] = RGB_values[2]
    return pic

def generate_semantic_im(RGB_image):
    new_obs = torch.tensor(RGB_image)
    new_obs = new_obs.permute(2,0,1).reshape(1,3,128,128)
    out,latent_space = model(new_obs)
    sample = out.cpu().argmax(dim=1)
    pic = replace(sample.numpy())
    return Image.fromarray(pic,'RGB')
